# Yahoo Groups message scraper

This notebookprovides code that will dowload messages from *private* Yahoo Groups. Another notebook `ScrapeBuild.jpynb` uses  [reqeusts](http://docs.python-requests.org/en/latest/user/install/). This notebook uses [Selenium](http://selenium-python.readthedocs.io/).


In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import datetime
import time
import sys

In [2]:
# Get groupname from user.
grp_name = input(prompt='What is the group name you seek to scape?  ')
# Give grp_name a default if no input.
if grp_name == '':
    grp_name = 'concatenative'

What is the group name you seek to scape?  backwoodspaintball


In [3]:
browser = webdriver.Chrome()
discarded_wait = input('Visit browser, log in credentials, press enter here when ready to continue.')

Visit browser, log in credentials, press enter here when ready to continue.


In [4]:
# Define a list to log successful attempts & error counter.
fetch_log = []
error_count = 0

# Iterate through the private group messages. The most recent group message number 
# can be found at: https://groups.yahoo.com/api/v1/groups/<groupname>/messages?count=10
# Parse the results to discern the most recent message number.
for i in range(120,100,-1):
    fetch_log.append(
        ('{} Working on message {}. Going backwards from 32811.'.format(
        str(datetime.datetime.now()), str(i))))
    try:
        browser.get(
            ''.join(
                (r'https://groups.yahoo.com/api/v1/groups/', grp_name, r'/messages/', str(i), r'/')))
        the_source = browser.page_source
        a_file = open(r'msgs\'' + grp_name + '_' + str(i) + '.html', 'w', encoding='utf-8')
        a_file.write(the_source)
        a_file.close()

        the_text = the_source[the_source.find(r'{'):-20]
        b_file = open(r'msgs\msg_no_' + str(i) + '.txt', 'w', encoding='utf-8')
        b_file.write(the_text)
        b_file.close()
    except:
        fetch_log.append(
            'Error loading and/or saving message {}. Error count is {}.'.format(i, error_count))
        error_count += 1
        if error_count > 100:
            sys.exit()

    # Optionally pause to assist in avoiding CAPTCHA and other anti-robot features.
    # time.sleep(1)

In [5]:
# Save fetch_log for later reference.
with open(
    ''.join((
        grp_name, '_fetch_log_', 
        str(datetime.datetime.now())[2:16].replace(" ", "-").replace(":",""), 
        '.log')), mode='w') as logfile:
            print('This is the log of fetched messages file from {}'.format(
                str(datetime.datetime.now())), file=logfile)
            print('Yahoo Group name {}.'.format(grp_name), file=logfile)
            for fetch_line in fetch_log:
                print(fetch_line, file = logfile)
logfile.close

<function TextIOWrapper.close>

In [6]:
import pandas as pd
import json
import datetime

In [7]:
# Define a list to hold structured data.
grandlist = []

for i in range(120,100,-1):
    # Define list to hold current record.
    observation = []
    raw_file = open(r'msgs\'' + grp_name + '_' + str(i) + '.html', 'r', encoding = 'utf-8')
    
    soup = BeautifulSoup(raw_file, 'html.parser')
    raw_json = soup.get_text()
    prs_msg = json.loads(raw_json, encoding = 'utf-8')
    
    # Add each variable to the current observation.
    observation.append(prs_msg['ygData']['userId'])
    observation.append(prs_msg['ygData']['authorName'])
    observation.append(str(prs_msg['ygData']['subject']).strip())
    observation.append(
        datetime.datetime.fromtimestamp(
            int(prs_msg['ygData']['postDate'])).strftime('%Y-%m-%d %H:%M:%S'))
    observation.append(prs_msg['ygData']['msgId'])
    observation.append(prs_msg['ygData']['numMessagesInTopic'])
    
    # Add current observation to grandlist.
    grandlist.append(observation)

In [8]:
pd_dataset = pd.DataFrame(grandlist, columns=
    ['userID', 'authorName', 'subject', 'postDate', 
     'msgId', 'numInTopic'])

In [9]:
# Check results.
pd_dataset.head()

,userID,authorName,subject,postDate,msgId,numInTopic
0,0,shadowsilence1,Re: Wow this site,2001-03-28 20:39:03,120,137
1,0,viper_rc1,Re: Wow this site,2001-03-27 19:22:21,119,137
2,0,jbossxc16,Wow this site,2001-03-26 18:51:31,118,137
3,0,mod70win,Re: tippman grenade,2001-03-20 20:07:39,117,137
4,0,jbossxc16,Re: Does anybody know...,2001-03-20 19:54:45,116,137
